In [ ]:
import pandas as pd
import random

import geopandas as gpd 
from shapely.geometry import Point, Polygon
import h3

import plotly.express as px

import json

In [27]:
# Cargar datos geoespaciales en un DataFrame GeoPandas
gdf = gpd.read_file('data/df_italia.geojson')
gdf.geometry = gdf.geometry.to_crs(epsg = 4326)
gdf.head()

,h3,COD_RIP,geometry
0,891f98a4027ffff,1,"POLYGON ((8.70271 45.72135, 8.70293 45.71959, ..."
1,891f9956aafffff,1,"POLYGON ((9.33777 45.41386, 9.33797 45.41208, ..."
2,891f9bb32d3ffff,1,"POLYGON ((9.10932 44.97942, 9.10952 44.97764, ..."
3,891f9c45d13ffff,1,"POLYGON ((7.02920 45.89853, 7.02945 45.89679, ..."
4,891f9b17517ffff,1,"POLYGON ((8.96436 44.73538, 8.96456 44.73360, ..."


In [ ]:
fig = px.choropleth_mapbox(gdf,
                           geojson=gdf.__geo_interface__,#['geometry']
                        #    locations=gdf.h3,
                        #    color='COD_RIP',
                        #    featureidkey='h3',
                           center={'lat': 40, 'lon':12},
                           mapbox_style='carto-positron',
                           zoom=6)
fig.show()

In [ ]:
import cbsodata

In [ ]:
geodata_url = 'https://cartomap.github.io/nl/wgs84/gemeente_2023.geojson'
municipal_boundaries = gpd.read_file(geodata_url)
municipal_boundaries.head()

In [ ]:
data = pd.DataFrame(cbsodata.get_data('83765NED', select = ['WijkenEnBuurten', 'Codering_3', 'GeboorteRelatief_25']))
data['Codering_3'] = data['Codering_3'].str.strip()
data.head()

In [ ]:
# Link data from Statistics Netherlands to geodata
municipal_boundaries = pd.merge(municipal_boundaries, data,
                               left_on = "statcode", 
                               right_on = "Codering_3")
municipal_boundaries.head()

In [24]:
#CRS: EPSG 3857 (web mercator projection wgs84)
municipal_boundaries.crs
# municipal_boundaries = municipal_boundaries.to_crs(epsg=3857)
municipal_boundaries = municipal_boundaries.to_crs(epsg=4326)

In [25]:
#First column: Geoid, geometry column and  data columns
gdf_choro = municipal_boundaries.copy()
gdf_choro['geoid'] = gdf_choro.index.astype(str)
gdf_choro = gdf_choro[['geoid', 'geometry', 'statnaam', 'GeboorteRelatief_25']]
gdf_choro.head(3)

,geoid,geometry,statnaam,GeboorteRelatief_25
0,0,"POLYGON ((6.77300 53.28300, 6.69800 53.25600, ...",Groningen,9
1,1,"POLYGON ((5.35100 52.40000, 5.30100 52.38200, ...",Almere,11
2,2,"POLYGON ((7.01500 53.07200, 7.03400 53.06500, ...",Stadskanaal,8


In [26]:
fig = px.choropleth_mapbox(gdf_choro,
                           geojson=gdf_choro.__geo_interface__,#['geometry']
                           locations=gdf_choro.geoid,
                           color='GeboorteRelatief_25',
                           featureidkey='properties.geoid',
                           center={'lat': 52.213, 'lon':5.2794},
                           mapbox_style='carto-positron',
                           zoom=6)
fig.show()

In [28]:
fig = px.choropleth_mapbox(gdf_choro,
                           geojson=gdf_choro.__geo_interface__,#['geometry']
                           locations=gdf_choro.geoid,
                           color='GeboorteRelatief_25',
                           featureidkey='properties.geoid',
                           center={'lat': 52.213, 'lon':5.2794},
                           mapbox_style='carto-positron',
                           zoom=6)

# Agregar evento de clic
fig.update_layout(
    clickmode='event+select',
    showlegend=False
)

def handle_click(trace, points, selector):
    if points.point_inds:
        selected_polygon = points.point_inds[0]
        print(f"Polígono seleccionado: {selected_polygon}")

fig.data[0].on_click(handle_click)

fig.show()

In [ ]:
with open('data/df_italia.geojson', 'r') as infile:
    geo_json = json.load(infile)